# Introduction

In this tutorial, we will go through an example to update a preexisting model. This might be useful when you come across additional data that you would want to consider, without having to train a model from scratch.

The main abstraction that Lightwood offers for this is the `BaseMixer.partial_fit()` method. To call it, you need to pass new training data and a held-out dev subset for internal mixer usage (e.g. early stopping). If you are using an aggregate ensemble, it's likely you will want to do this for every single mixer. The convienient `PredictorInterface.adjust()` does this automatically for you.


# Initial model training

First, let's train a Lightwood predictor for the `concrete strength` dataset:

In [1]:
from lightwood.api.high_level import ProblemDefinition, json_ai_from_problem, predictor_from_json_ai
import pandas as pd

ModuleNotFoundError: No module named 'lightwood'

In [ ]:
# Load data
df = pd.read_csv('https://raw.githubusercontent.com/mindsdb/lightwood/staging/tests/data/concrete_strength.csv')

df = df.sample(frac=1, random_state=1)
train_df = df[:int(0.2*len(df))]
update_df = df[int(0.2*len(df)):int(0.8*len(df))]
test_df = df[int(0.8*len(df)):]

print(f'Train dataframe shape: {train_df.shape}')
print(f'Update dataframe shape: {update_df.shape}')
print(f'Test dataframe shape: {test_df.shape}')

Note that we have three different data splits.

We will use the `training` split for the initial model training. As you can see, it's only a 20% of the total data we have. The `update` split will be used as training data to adjust/update our model. Finally, the held out `test` set will give us a rough idea of the impact our updating procedure has on the model's predictive capabilities.

In [ ]:
# Define predictive task and predictor
target = 'concrete_strength'
pdef = ProblemDefinition.from_dict({'target': target, 'time_aim': 200})
jai = json_ai_from_problem(df, pdef)

# We will keep the architecture simple: a single neural mixer, and a `BestOf` ensemble:
jai.outputs[target].mixers = [{
    "module": "Neural",
    "args": {
        "fit_on_dev": False,
        "stop_after": "$problem_definition.seconds_per_mixer",
        "search_hyperparameters": False,
    }
}]

jai.outputs[target].ensemble = {
    "module": "BestOf",
    "args": {
        "args": "$pred_args",
        "accuracy_functions": "$accuracy_functions",
    }
}

# Build and train the predictor
predictor = predictor_from_json_ai(jai)
predictor.learn(train_df)

In [ ]:
# Train and get predictions for the held out test set
predictions = predictor.predict(test_df)
predictions

## Updating the predictor

As previously mentioned, you can update any given mixer with a `BaseMixer.partial_fit()` call. If you have multiple mixers and want to update them all at once, you should use `PredictorInterface.adjust()`. 

For both of these methods, two encoded datasources are needed as input (for `adjust` you need to wrap them in a dictionary with 'old' and 'new' keys). 

Let's `adjust` our predictor:

In [ ]:
from lightwood.data import EncodedDs

train_ds = EncodedDs(predictor.encoders, train_df, target)
update_ds = EncodedDs(predictor.encoders, update_df, target)

predictor.adjust(update_ds, train_ds)  # <Data to adjust, Old Data>

In [ ]:
new_predictions = predictor.predict(test_df)
new_predictions

Nice! Our predictor was updated, and new predictions are looking good. Let's compare the old and new accuracies:

In [ ]:
from sklearn.metrics import r2_score
import numpy as np

old_acc = r2_score(test_df['concrete_strength'], predictions['prediction'])
new_acc = r2_score(test_df['concrete_strength'], new_predictions['prediction'])

print(f'Old Accuracy: {round(old_acc, 3)}\nNew Accuracy: {round(new_acc, 3)}')

After updating, we see an increase in the R2 score of predictions for the held out test set.

## Conclusion

We have gone through a simple example of how Lightwood predictors can leverage newly acquired data to improve their predictions. The interface for doing so is fairly simple, requiring only some new data and a single call to update.

You can further customize the logic for updating your mixers by modifying the `partial_fit()` methods in them.